# Customer Analysis - Explore Customer Behavior

## Import

Needed packages. Pyspark will be used for data managing and plotly for visualisations. Keep in mind to install
JAVA so Spark will work properly.

Used dataset is from https://rees46.com/de found on https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store.

In [2]:
# %%timeit
import os
import pyspark
import pandas as pd
import pyspark.sql.functions as f
import plotly.express as px
import plotly.graph_objects as go

## Read

The data needs to be located in ```data/``` in unzipped form as a csv.

In [3]:
# %%timeit -n 1

# spark = pyspark.sql.SparkSession.builder.appName("app1").getOrCreate()
spark = pyspark.sql.SparkSession \
    .builder \
    .appName("app_great") \
    .master("local") \
    .config("spark.executor.memory", f"16g") \
    .config("spark.driver.memory", f"16g") \
    .config("spark.memory.offHeap.enabled", True) \
    .config("spark.memory.offHeap.size", f"16g") \
    .config("spark.sql.debug.maxToStringFields", f"16") \
    .config("spark.driver.maxResultSize", f"4g") \
    .getOrCreate()
sc = spark.sparkContext

In [4]:
# %%timeit

# read raw data
sdf_201911 = spark.read.csv("data/2019-Nov.csv", header=True, inferSchema=True)
sdf_201910 = spark.read.csv("data/2019-Oct.csv", header=True, inferSchema=True)

# join both months together
sdf = sdf_201910.union(sdf_201911)

In [5]:
# %%timeit

# sdf = spark.read.csv("data/test_data.csv", header=True, inferSchema=True)

In [6]:
# %%timeit

sdf.show(3)

+--------------------+----------+----------+-------------------+--------------------+--------+-----+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+-----+---------+--------------------+
|2019-10-01 00:00:...|      view|  44600062|2103807459595387724|                null|shiseido|35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:...|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua| 33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:...|      view|  17200506|2053013559792632471|furniture.living_...|    null|543.1|519107250|566511c2-e2e3-422...|
+--------------------+----------+----------+-------------------+--------------------+--------+-----+---------+--------------------+
only showing top 3 rows



## Preparation

Prepare and enhance data for analysis and modelling.

In [7]:
# %%timeit

# Datatypes
sdf = sdf.withColumn("event_time", sdf["event_time"].cast(pyspark.sql.types.TimestampType()))
sdf = sdf.withColumn("category_id", sdf["category_id"].cast(pyspark.sql.types.StringType()))
sdf = sdf.withColumn("product_id", sdf["product_id"].cast(pyspark.sql.types.StringType()))
sdf = sdf.withColumn("user_id", sdf["user_id"].cast(pyspark.sql.types.StringType()))

In [8]:
# %%timeit

# Feature Splitting
# sdf = sdf.withColumn("category_class", f.substring_index(sdf.category_code, '.', 1))

sdf = sdf.withColumn("category_class", f.split(sdf["category_code"], "\.").getItem(0))
sdf = sdf.withColumn("category_sub_class", f.split(sdf["category_code"], "\.").getItem(1))
sdf = sdf.withColumn("category_sub_sub_class", f.split(sdf["category_code"], "\.").getItem(2))

sdf = sdf.withColumn("year", f.year("event_time"))
sdf = sdf.withColumn("month", f.month("event_time"))
sdf = sdf.withColumn("weekofyear", f.weekofyear("event_time"))
sdf = sdf.withColumn("dayofyear", f.dayofyear("event_time"))
sdf = sdf.withColumn("dayofweek", f.dayofweek("event_time"))
sdf = sdf.withColumn("dayofmonth", f.dayofmonth("event_time"))
sdf = sdf.withColumn("hour", f.hour("event_time"))

sdf = sdf.withColumn('turnover', f.when(f.col('event_type') == 'purchase', f.col('price')).otherwise(0))
sdf = sdf.withColumn('bougth_quantity', f.when(f.col('event_type') == 'purchase', f.lit(1)).otherwise(0))
sdf = sdf.withColumn('viewed_quantity', f.when(f.col('event_type') == 'view', f.lit(1)).otherwise(0))
sdf = sdf.withColumn('cart_quantity', f.when(f.col('event_type') == 'cart', f.lit(1)).otherwise(0))
# None Handling
# sdf = sdf.fillna(value="not defined")

sdf.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)
 |-- category_class: string (nullable = true)
 |-- category_sub_class: string (nullable = true)
 |-- category_sub_sub_class: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- weekofyear: integer (nullable = true)
 |-- dayofyear: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- turnover: double (nullable = true)
 |-- bougth_quantity: integer (nullable = false)
 |-- viewed_quantity: integer (nullable = false)
 |-- cart_quantity: integer (nullable = false)



## Dataframe Creation

create several dataframes with different aggregation level to answer different questions/ tasks.

In [9]:
# %%timeit

# raw
sdf_raw = sdf

In [10]:
# %%timeit

sdf.createOrReplaceTempView("Data")

## Field Explanations

Following fields are in the standard dataset:
- event_time
- product_id
- category_id
- category_code
- brand
- price
- user_id
- user_session

### General
In this overview you can see the count of unique rows, product_ids, category_classes, category_codes, category_ids, brands, user_ids and user_sessions as well as the average price of the products.

In [11]:
# %%timeit

sdf_count_overview = spark.sql("SELECT COUNT(*) AS Row_Count, \
                                       COUNT(DISTINCT(product_id)) AS Product_ID, \
                                       COUNT(DISTINCT(category_class)) AS Category_Class, \
                                       COUNT(DISTINCT(category_code)) AS Category_Code, \
                                       COUNT(DISTINCT(category_id)) AS Category_ID, \
                                       COUNT(DISTINCT(brand)) AS Brand, \
                                       COUNT(DISTINCT(user_id)) AS User_ID, \
                                       COUNT(DISTINCT(user_session)) AS User_Session, \
                                       ROUND(MEAN(price),2) AS AVG_Price \
                                FROM Data")
sdf_count_overview.show()

+---------+----------+--------------+-------------+-----------+-----+-------+------------+---------+
|Row_Count|Product_ID|Category_Class|Category_Code|Category_ID|Brand|User_ID|User_Session|AVG_Price|
+---------+----------+--------------+-------------+-----------+-----+-------+------------+---------+
|109950743|    206876|            13|          129|        691| 4303|5316649|    23016650|   291.63|
+---------+----------+--------------+-------------+-----------+-----+-------+------------+---------+



In [ ]:
# %%timeit

sdf_raw.select("product_id", "category_class", "category_code", "category_id", "brand", "user_id", "user_session", "price").describe().show()

In [ ]:
# %%timeit

sdf_raw.show(1, vertical=True)


### event_time

In [ ]:
# %%timeit

sdf_raw.select("event_time").show(5)


### event_type
The event_type describes the kind of interaction, an user had with a product. The field can take three forms: View, Cart and Purchase. The distribution of these three forms is represented in the following plot:

In [ ]:
# %%timeit

sdf_event_type_dist = sdf_raw.groupBy("event_type").count()
sdf_event_type_dist.show()

In [ ]:
# %%timeit

# Plot Event Types
df = sdf_event_type_dist.select("count", "event_type").toPandas()
fig = px.pie(df, values='count', names='event_type', title='Distribution of Customer Actions')
fig.write_html("data/exports/Distribution of Customer Actions.html")
fig.show()

### product_id
The product_id is the unique identificator for a product. As you can see in the overview, there are ... unique product_ids in the datasets Oct-2019 and Nov-2019, the users have interacted with.


In [ ]:
# %%timeit


sdf_count_per_product_id = sdf_raw.groupBy("product_id").count().orderBy(f.desc("count"))

px.bar(sdf_count_per_product_id.limit(10).toPandas(), x='product_id', y='count', title="Top 10 most interacted products")


### category_id
The category_id is an unique identifier for the category of a Product. Every Product is assigned to a single category_id, which is summarizing many product_ids into categories. This knowledge is based on the more detailed analyzes within the file "product_analysis.ipnynb". As you can see in the overview, there are ... unique category_ids.

In [ ]:
# %%timeit

sdf_count_per_category_id = sdf_raw.groupBy("category_id").count().orderBy(f.desc("count"))


px.bar(sdf_count_per_category_id.limit(10).toPandas(), x="category_id", y="count", title="Top 10  category_ids most interacted with")

### category_code
The category_code is describing the category, a product_id and category_id is assigned to. Every Product_id and Category_id is assigned to a single category_code, which is summarizing many product_ids and category_ids into categories. This knowledge is also based on the more detailed analyzes within the file "product_analysis.ipnynb". As you can see in the overview, there are ... unique category_code.

In [ ]:
# %%timeit

sdf_count_per_category_code = sdf_raw.groupBy("category_code").count().orderBy(f.desc("count"))


px.bar(sdf_count_per_category_code.limit(10).toPandas(), x="category_code", y="count", title="Top 10 category_code most interacted with")

### brand
The brand indicates the brand of a product_id. It is independent of the categories, so that a brand can appear in many category_classes. This knowledge is also based on the more detailed analyzes within the file "product_analysis.ipnynb". There are ... unique brands in the dataset. Thereby you can see the most popular brands in the following plot:

In [ ]:
# %%timeit

sdf_count_per_brand = sdf_raw.groupBy("brand").count().orderBy(f.desc("count"))

px.histogram(sdf_count_per_brand.limit(10).toPandas(), x="brand", y="count", title="Top 10 brands most interacted with")

### price

In [ ]:
# %%timeit

sdf_raw.describe("price").show()

In [ ]:
# %%timeit

sdf_avg_prices = sdf_raw.select("product_id", "price").distinct()
px.box(sdf_avg_prices.toPandas(), y="price", title="Prices for every product")


### user_id

In [ ]:
# %%timeit

sdf_raw.select("user_id").show(5)

In [ ]:
# %%timeit

# creating spark dataframe for unique number of users each month

sdf_user_id_by_month = sdf_raw.select("user_id", "month").distinct().groupBy("month").count()

In [ ]:
# %%timeit

# Data Prep for user graph

pdf_usr_id_by_mnth = sdf_user_id_by_month.toPandas()
pdf_usr_id_by_mnth = pdf_usr_id_by_mnth[pdf_usr_id_by_mnth.month != 12]
pdf_usr_id_by_mnth.month = pdf_usr_id_by_mnth.month.astype(str)
px.bar(pdf_usr_id_by_mnth, x="month", y="count", title="Unique users each month")

In October 3.02 million people visited the site and in November the total user count grew to 3.696 million. Which nets
a difference of around 700,000 users.

In [ ]:
# %%timeit

sdf_user_id_both_months = sdf_raw.select("user_id", "month").distinct().groupby("user_id").sum()

cnt = sdf_user_id_both_months.where(sdf_user_id_both_months["sum(month)"] == 21).count()
print("Amount of users that visited the site in both months:", cnt)

Not all users in October and November are to be expected to have only visited in the respective month.
A total of 1,400,979 users visited the page in both October and November.

### user_session

In [ ]:
# %%timeit

# avg actions per session
sdf_cnt_action_per_session = sdf_raw.groupby("user_session").count()
sdf_cnt_action_per_session.describe().show()

## Feature Enginnering

### category_class and category_sub_class
The category_code consists of two or three parts in general, which are separeted by a dot. A possible category_code is for example: appliances.kitchen.washer or electronics.smartphone. Because of that the category code can be splited into to the categories: category_class, category_sub_class and category_sub_sub_class.

The category_class is representing the fist part of the category_code. It can be used to summarize many category_codes into an overarching category_class. As you can see in the overview, there are ... unique category_classes.

In [ ]:
# %%timeit

sdf_agg_classes = sdf_raw.groupBy("category_class", "category_sub_class", "category_sub_sub_class").count().na.fill(value="not defined")
sdf_agg_classes = sdf_agg_classes.where(sdf_agg_classes["category_class"] != "not defined")
sdf_agg_classes.show()

In [ ]:
# %%timeit

px.sunburst(sdf_agg_classes.toPandas(), path=["category_class", "category_sub_class", "category_sub_sub_class"], values="count", title="Category Classes and Subclasses (without not defined data in category_class)")

### Time - Fields

The column “event_time” allows you to create the following columns: year, month, weekofyear, dayofyear, dayofweek, dayofmonth and hour. These columns allow advanced analysis.

- Year: The Year-column contains only the year 2019 since the dataset only covers this year.
- Month: The month-column cotains the values 10, 11 and 12,  which are representing the october, november and december 2019.DeprecationWarning
- Weekofyear: The weekofyear-column is covering the weeks ... - ...
- Dayofyear: The dayofyear-column is covering the days .... - ...
- Dayofweek: The dayofweek-column is covering the values 1-7. These values are representing days sunday(1), monday(2), Tuesday(3), Wednesday(4), Thursday(5), Friday(6) and Saturday(7).
- Dayofmonth: The dayofmonth-column contains the values 1-31, which are reperesenting the day in the corresponding month.
- hour: The hour-column contains the values 0-24, which are representing the hour of interaction.

### Turnover and quantities

In the following table you can see that in some cases a product ID was purchased several times per user session.

In [ ]:
# %%timeit

sdf_count_session_product = sdf_raw.where(sdf_raw["event_Type"] == 'purchase').groupBy("user_session", "product_id", "event_type").count()

sdf_count_session_product.show()

From this it can be concluded that a single interaction is created for each product purchased. With the help of this information, the columns "turnover", "bought_quantity", "viewed_quantity", "cart_quantity" can be created.

- Turnover: The turnover is equivalent to the price, if the event_type is equal to "purchase".
- bought_quantity: The bought_quantity describes the quantity of a product,that had been bought. In the unaggregated form it only contains the values 0 and 1.
- viewed_quantity: The viewed_quantity describes the quantity of a product,that had been viewed. In the unaggregated form it only contains the values 0 and 1.
- cart_quantity: The cart_quantity describes the quantity of a product,that had been put into the cart. In the unaggregated form it only contains the values 0 and 1.

These columns are particularly suitable for aggregated analyses.

In [ ]:
# %%timeit

sdf_raw.show()

In [ ]:
# %%timeit

sdf_count_turnover_quantity = sdf_raw.agg(f.sum("turnover"), f.sum("bougth_quantity"), f.sum("viewed_quantity"), f.sum("Cart_quantity"))
sdf_count_turnover_quantity.show()

## Exploration and Analysis

### Time Distribution

#### Time and Events

In [ ]:
# %%timeit

# aggregated time (weeks, dayofweeks, month)

sdf_time_dist_month = sdf.groupBy("event_type", "dayofmonth").count()
sdf_time_dist_month = sdf_time_dist_month.withColumnRenamed("count", "cnt")
sdf_time_dist_month = sdf_time_dist_month.sort("dayofmonth", "event_type")
sdf_time_dist_week = sdf.groupBy("event_type", "dayofweek").count()
sdf_time_dist_week = sdf_time_dist_week.withColumnRenamed("count", "cnt")
sdf_time_dist_week = sdf_time_dist_week.sort("dayofweek", "event_type")

sdf_time_dist_day = sdf.groupBy("event_type", "hour").count()
sdf_time_dist_day = sdf_time_dist_day.withColumnRenamed("count", "cnt")
sdf_time_dist_day = sdf_time_dist_day.sort("hour", "event_type")

In [ ]:
# %%timeit

# Timestamp Distribution (per event_type) over every day of month

df = sdf_time_dist_month.toPandas()

fig = px.bar(df, x = 'dayofmonth', y = 'cnt', color ='event_type', barmode = 'stack')

fig.update_layout(title = "Number of events over a month",
     xaxis_title = 'Day of Month', yaxis_title = 'Number of Events')
fig.update_xaxes(type="category")
fig.write_html("data/exports/Number of events over a month.html")
fig.show()

In [ ]:
# %%timeit

# Timestamp Distribution (per event_type) over every day of week

df = sdf_time_dist_week.toPandas()

fig = px.bar(df, x = 'dayofweek', y = 'cnt', color ='event_type', barmode = 'stack')

fig.update_layout(title = "Number of events over a week",
     xaxis_title = 'Day of Week', yaxis_title = 'Number of Events')
fig.update_xaxes(type="category")
fig.write_html("data/exports/Number of events over a week.html")
fig.show()

In [ ]:
# %%timeit

# Timestamp Distribution (per event_type) over every hour of a day

df = sdf_time_dist_day.toPandas()

fig = px.bar(df, x = 'hour', y = 'cnt', color ='event_type', barmode = 'stack')

fig.update_layout(title = "Number of events over a day",
     xaxis_title = 'Hour of day', yaxis_title = 'Number of Events')
fig.update_xaxes(type="category")
fig.write_html("data/exports/Number of events over a day.html")
fig.show()

#### Time and Turnover

In [ ]:
# %%timeit


sdf_month_Umsatz = sdf_raw.groupBy("month", "dayofmonth").sum("turnover").orderBy(f.asc("month"), f.asc("dayofmonth"))

In [ ]:
# %%timeit

sdf_month_Umsatz = sdf_month_Umsatz.withColumn("month", sdf_month_Umsatz["month"].cast(pyspark.sql.types.StringType()))


In [ ]:
# %%timeit

df = sdf_month_Umsatz.toPandas()

fig = px.bar(df, x = "dayofmonth", y = "sum(turnover)", color ='month', barmode = 'stack')

fig.update_layout(title = "Turnover per Month",
     xaxis_title = 'Day of Month', yaxis_title = 'Turnover')
fig.update_xaxes(type="category")
fig.write_html("data/exports/Turnover per Month.html")
fig.show()

In [ ]:
# %%timeit

sdf_week_Umsatz = sdf_raw.groupBy("weekofyear", "dayofweek").sum("turnover").orderBy(f.asc("weekofyear"), f.asc("dayofweek"))

In [ ]:
# %%timeit

sdf_week_Umsatz = sdf_week_Umsatz.withColumn("weekofyear", sdf_week_Umsatz["weekofyear"].cast(pyspark.sql.types.StringType()))


In [ ]:
# %%timeit

df = sdf_week_Umsatz.toPandas()

fig = px.bar(df, x = "dayofweek", y = "sum(turnover)", color ='weekofyear', barmode = 'stack')

fig.update_layout(title = "Turnover per Week",
     xaxis_title = 'Day of Week', yaxis_title = 'Turnover')
fig.update_xaxes(type="category")
fig.write_html("data/exports/Turnover per Week.html")
fig.show()

KeyboardInterrupt: 

### Category and products

#### Connection between category_class, category_code, category_id, product_id and brand

Connection between category_class, category_code, category_id, product_id and brand
The product_id is a subset of the category_id, which is a subset of the category_code. The category_code is in turn a subset of the category_class. (product_id ⊂ category_id ⊂ category_code ⊂ category_class). The brand on the otherhand is cross-class. This knowledge is based on the more detailed analyzes within the file "product_analysis.ipnynb".

In the following plot you can see distribution of the product_id, category_id and category_code within the category_class. It´s possible to access a more detailed view by selecting a special category_class, category_code or category_id.

In [ ]:
# %%timeit

sdf_product_per_category = sdf_raw.groupBy("category_id").agg(f.countDistinct("product_id"))

df = sdf_product_per_category.toPandas()
px.box(df, y="count(product_id)", title="Number of products per category_id")

In [ ]:
# %%timeit

sdf_agg_brand_category = sdf_raw.groupBy("category_class", "brand", "product_id").count().na.drop()
px.sunburst(sdf_agg_brand_category.toPandas(), path=["category_class", "brand"], values="count", title="Brands per Category_class (without not defined data")

#### Connection to the price

The following plots will represent the price distribution within the category_classes, category_codes, category_ids, product_ids and brands.

In [ ]:
# %%timeit

px.box(sdf_raw.select("category_class", "price").toPandas(), x="category_class", y="price", title="Price ~ Category_class")

In [ ]:
# %%timeit

sdf_price_per_product = sdf_raw.select("product_id", "price").distinct().orderBy(f.desc("price"))

sdf_price_per_product.show(10)

#### Connection to the event-type
The following plots will represent the event_type distribution within the category_classes, category_codes, category_ids, product_ids and brands.

In [ ]:
# %%timeit

sdf_category_class_event_distribution = sdf_raw.groupBy("category_class", "event_type").count().na.drop()

px.sunburst(sdf_category_class_event_distribution.toPandas(), path=['category_class','event_type'], values="count", title="Event Type per Category_Class (no None Values)")

#### Connection to the turnover

In [ ]:
# %%timeit

sdf_brand_overview = sdf_raw.groupBy("brand").agg(f.sum("turnover"), f.avg("price"), f.sum("viewed_quantity"), f.sum("cart_quantity"), f.sum("bougth_quantity")).orderBy(f.desc("sum(turnover)"))
sdf_brand_overview = sdf_brand_overview.withColumn("sum(turnover)", f.round(sdf_brand_overview["sum(turnover)"], 2))
sdf_brand_overview = sdf_brand_overview.withColumn("avg(price)", f.round(sdf_brand_overview["avg(price)"], 2))

sdf_brand_overview.show(10)


In [ ]:
# %%timeit

sdf_category_code_overview = sdf_raw.groupBy("category_code").agg(f.sum("turnover"), f.avg("price"), f.sum("viewed_quantity"), f.sum("cart_quantity"), f.sum("bougth_quantity")).orderBy(f.desc("sum(turnover)"))
sdf_category_code_overview = sdf_category_code_overview.withColumn("sum(turnover)", f.round(sdf_category_code_overview["sum(turnover)"], 2))
sdf_category_code_overview = sdf_category_code_overview.withColumn("avg(price)", f.round(sdf_category_code_overview["avg(price)"], 2))

sdf_category_code_overview.show(10)



In [ ]:
# %%timeit

sdf_category_class_overview = sdf_raw.groupBy("category_class").agg(f.sum("turnover"), f.avg("price"), f.sum("viewed_quantity"), f.sum("cart_quantity"), f.sum("bougth_quantity")).orderBy(f.desc("sum(turnover)"))
sdf_category_class_overview = sdf_category_class_overview.withColumn("sum(turnover)", f.round(sdf_category_class_overview["sum(turnover)"], 2))
sdf_category_class_overview = sdf_category_class_overview.withColumn("avg(price)", f.round(sdf_category_class_overview["avg(price)"], 2))

sdf_category_class_overview.show(10)


### Event_Type and Price
The following plot represents the distribution of the price within the event_type.

### User analysis

Overview:
- How many sessions does a user have on average?
- How many products does a user buy / view / put in cart on average?
- How many interactions does a user have within one session on average?
- Avrg Sessions per (Week)Day => Boxplot?


#### How many sessions does a user have on average?

In [ ]:
# %%timeit

sdf_agg_user = sdf_raw.select("user_id", "user_session").distinct().groupBy("user_id").count()
sdf_agg_user.select("count").describe().show()

In [ ]:
# %%timeit

print("Statistical distribution of Sessions per User in October")
sdf_agg_user_mnth = sdf_raw.select("user_id", "user_session", "month").distinct().groupBy("user_id", "month").count()
sdf_agg_user_mnth.where(sdf_agg_user_mnth.month == 10).select("count").describe().show()

print("Statistical distribution of Sessions per User in November")
sdf_agg_user_mnth = sdf_raw.select("user_id", "user_session", "month").distinct().groupBy("user_id", "month").count()
sdf_agg_user_mnth.where(sdf_agg_user_mnth.month == 11).select("count").describe().show()

#### How many products does a user buy / view / put in cart on average?

In [ ]:
# %%timeit

sdf_amount_events = sdf_raw.select("user_id", "event_type").groupBy("event_type").count()
amount_usr = sdf_raw.select("user_id").distinct().count()
pdf_avrg_events_usr = sdf_amount_events.toPandas()

pdf_avrg_events_usr["count"] = pdf_avrg_events_usr["count"].div(amount_usr)
print(pdf_avrg_events_usr)

A user views on average XX item, puts XX items in their cart and buys XX items.


In [ ]:
# %%timeit

sdf_agg_user_type = sdf_raw.select("user_id", "event_type").groupBy("user_id").count()
most_active_user = sdf_agg_user_type.sort("count", ascending=False).take(1)

print(f"The most active user has the ID {most_active_user[0][0]} with a total of {most_active_user[0][1]} interactions.")

#### How many interactions does a user have within one session on average? - done

In [ ]:
# %%timeit

sdf_amount_interactions_per_sess = sdf_raw.select("user_session", "event_type").groupBy("user_session", "event_type").count()
# sdf_amount_interactions_per_sess.show()
print("Average amount of views per session:")
sdf_amount_interactions_per_sess.where(sdf_amount_interactions_per_sess.event_type == "view").agg({"count":"mean"}).show()
print("Average amount of 'Add to Cart' per session:")
sdf_amount_interactions_per_sess.where(sdf_amount_interactions_per_sess.event_type == "cart").agg({"count":"mean"}).show()
print("Average amount of purchases per session:")
sdf_amount_interactions_per_sess.where(sdf_amount_interactions_per_sess.event_type == "purchase").agg({"count":"mean"}).show()

#### Avrg Sessions per week day

In [ ]:
# %%timeit

# User session distribution over every day of week

sdf_usr_ses_dist_week = sdf_raw.select("user_session", "dayofyear", "dayofweek").groupBy("user_session","dayofyear","dayofweek").count()
pdf_usr_sess_time_dist = sdf_usr_ses_dist_week.toPandas()
fig_usr_time = px.box(pdf_usr_sess_time_dist, x="dayofweek", y="Session", title="Average Sessions per Weekday")
fig_usr_time.update_layout(
    xaxis_title="Day of Week"
)

### Unique sessions each month

In [ ]:
# %%timeit

sdf_ses_by_mnth = sdf_raw.select("user_session", "month").distinct().groupby("month").count()

pdf_usr_ses_by_mnth = sdf_ses_by_mnth.toPandas()
pdf_usr_ses_by_mnth = pdf_usr_ses_by_mnth[pdf_usr_ses_by_mnth.month != 12]
pdf_usr_ses_by_mnth.month = pdf_usr_id_by_mnth.month.astype(str)
px.bar(pdf_usr_id_by_mnth, x="month", y="count", title="Unique sessions each month")

#### User and turnover

In [ ]:
# %%timeit

sdf_user_overview = sdf_raw.groupBy("user_id").agg(f.sum("turnover"), f.count("user_session"), f.sum("viewed_quantity"), f.sum("cart_quantity"), f.sum("bougth_quantity")).orderBy(f.desc("sum(turnover)"))
sdf_user_overview.show(10)

### session analysis



- number of events
- session_start_time
- session_stop_time
- session_success
- products_bought
- products_viewed
- turnover


In [ ]:
# %%timeit

sdf_session = sdf.select("user_id", "user_session", "event_type", "product_id", "price", "event_time") #.orderBy("user_id", "user_session")

In [ ]:
# %%timeit

sdf_session = sdf_session.withColumn("views", f.when(sdf_session.event_type == "view", 1).otherwise(0))
sdf_session = sdf_session.withColumn("purchases", f.when(sdf_session.event_type == "purchase", 1).otherwise(0))
sdf_session = sdf_session.withColumn("carts", f.when(sdf_session.event_type == "cart", 1).otherwise(0))
sdf_session = sdf_session.withColumn("turnover", f.when(sdf_session.event_type == "purchase", sdf_session["price"]).otherwise(0))

sdf_session = sdf_session.withColumn("first_event", sdf_session.event_time)
sdf_session = sdf_session.withColumn("last_event", sdf_session.event_time)

In [ ]:
# %%timeit

sdf_session_agg = sdf_session.groupBy("user_id", "user_session").agg(f.sum("turnover"), f.sum("views"), f.sum("purchases"), f.sum("carts"), f.min("event_time"), f.max("event_time"))
sdf_session_agg = sdf_session_agg.withColumn("duration", (sdf_session_agg["max(event_time)"] - sdf_session_agg["min(event_time)"]))
sdf_session_agg = sdf_session_agg.withColumn("sum(events)", (sdf_session_agg["sum(views)"] + sdf_session_agg["sum(purchases)"] + sdf_session_agg["sum(carts)"]))
sdf_session_agg = sdf_session_agg.withColumn("turnover", f.when(sdf_session_agg["sum(purchases)"] > 0, (sdf_session_agg["sum(purchases)"] *  sdf_session_agg["sum(turnover)"])).otherwise(0))

sdf_session_agg = sdf_session_agg.withColumn("successfull", f.when(sdf_session_agg["sum(purchases)"] > 0, 1).otherwise(0))

In [ ]:
# %%timeit

sdf_session_agg.printSchema()

In [ ]:
# %%timeit

sdf_session_agg.show()

### Customer Profiles

In preparation for clustering a customer profile will be created:

- customer_id
- number_of_view_events
- number_of_cart_events
- number_of_purchase_events
- total_turnover
- number_of_bought_items (resolve multiple purchasing events for quantity)
- avg_sold_cart
- avg_session_time
- avg_actions_per_session



In [ ]:
# %%timeit

# time period simply dependend on input data
sdf_customer_profile = sdf_session_agg.groupBy("user_id").agg(f.sum("sum(events)"), f.sum("sum(views)"), f.sum("sum(purchases)"), f.sum("sum(carts)"), f.sum("turnover"), f.count("user_session"), f.sum("successfull"))

sdf_customer_profile = sdf_customer_profile.withColumn("avg_turnover_per_session", (sdf_customer_profile["sum(turnover)"] / sdf_customer_profile["count(user_session)"]))
sdf_customer_profile = sdf_customer_profile.withColumn("avg_events_per_session", (sdf_customer_profile["sum(sum(events))"] / sdf_customer_profile["count(user_session)"]))

In [ ]:
# %%timeit

sdf_customer_profile.show()

### Correlation matrix

#### Daytime - Correlation Matrix

In [ ]:
# %%timeit

sdf_corr_time = sdf_raw.select("event_time", "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity")
sdf_corr_time = sdf_corr_time.withColumn("hour", f.hour("event_time"))

In [ ]:
# %%timeit

sdf_corr_time = sdf_corr_time.withColumn('Morning', f.when((f.col('hour')>=6) & (f.col('hour')<12), f.lit(1)).otherwise(0))
sdf_corr_time = sdf_corr_time.withColumn('Afternoon', f.when((f.col('hour')>=12) & (f.col('hour')<18), f.lit(1)).otherwise(0))
sdf_corr_time = sdf_corr_time.withColumn('Evening', f.when(f.col('hour') > 18 , f.lit(1)).otherwise(0))
sdf_corr_time = sdf_corr_time.withColumn('Night', f.when(f.col('hour') < 6, f.lit(1)).otherwise(0))

In [ ]:
# %%timeit

sdf_corr_time = sdf_corr_time.select("Morning", "Afternoon", "Evening", "Night", "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity")

In [ ]:
# %%timeit

sdf_corr_time.toPandas().corr().style.background_gradient(cmap='bwr')

#### Weekday - Correlation Matrix

In [ ]:
# %%timeit

sdf_corr_dayofweek = sdf_raw.select("dayofweek", "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity")

In [ ]:
# %%timeit

# One-hot-encoding
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Sunday', f.when(f.col('dayofweek') == '1', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Monday', f.when(f.col('dayofweek') == '2', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Tuesday', f.when(f.col('dayofweek') == '3', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Wednesday', f.when(f.col('dayofweek') == '4', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Thursday', f.when(f.col('dayofweek') == '5', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Friday', f.when(f.col('dayofweek') == '6', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Saturday', f.when(f.col('dayofweek') == '7', f.lit(1)).otherwise(0))

In [ ]:
# %%timeit

sdf_corr_dayofweek = sdf_corr_dayofweek.select("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday",  "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity" )

In [ ]:
# %%timeit

sdf_corr_dayofweek.toPandas().corr().style.background_gradient(cmap='bwr')

#### Category Class - Correlation Matrix

In [ ]:
# %%timeit

sdf_corr_category_class = sdf_raw.select("category_class", "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity")

In [ ]:
# One-hot-encoding
sdf_corr_category_class = sdf_corr_category_class.withColumn('computers', f.when(f.col('category_class') == 'computers', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('auto', f.when(f.col('category_class') == 'auto', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('apparel', f.when(f.col('category_class') == 'apparel', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('appliances', f.when(f.col('category_class') == 'appliances', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('furniture', f.when(f.col('category_class') == 'furniture', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('accessories', f.when(f.col('category_class') == 'accessories', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('electronics', f.when(f.col('category_class') == 'electronics', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('construction', f.when(f.col('category_class') == 'construction', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('not_defined', f.when(f.col('category_class') == 'not defined', f.lit(1)).otherwise(0))

In [ ]:
# %%timeit

sdf_corr_category_class = sdf_corr_category_class.select("computers", "auto", "apparel", "appliances", "furniture", "accessories", "electronics", "construction", "not_defined", "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity" )

In [ ]:
# %%timeit

sdf_corr_category_class.toPandas().corr().style.background_gradient(cmap='bwr')

#### Month - Correlation Matrix

In [ ]:
# %%timeit

sdf_corr_month = sdf_raw.select("dayofmonth", "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity")

In [ ]:
# %%timeit

# One-hot-encoding
sdf_corr_month = sdf_corr_month.withColumn('Beginningofmonth', f.when(f.col('dayofmonth')<10, f.lit(1)).otherwise(0))
sdf_corr_month = sdf_corr_month.withColumn('Middleofmonth', f.when((f.col('dayofmonth')>=10) & (f.col('dayofmonth')<20), f.lit(1)).otherwise(0))
sdf_corr_month = sdf_corr_month.withColumn('Endofmonth', f.when(f.col('dayofmonth') > 20 , f.lit(1)).otherwise(0))

In [ ]:
# %%timeit

sdf_corr_month = sdf_corr_month.select("Beginningofmonth", "Middleofmonth", "Endofmonth", "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity" )

In [ ]:
# %%timeit

sdf_corr_month.toPandas().corr().style.background_gradient(cmap='bwr')

#### Price - Correlation Matrix

In [ ]:
# %%timeit

sdf_corr_price = sdf_raw.select("price", "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity")
sdf_corr_price.toPandas().corr().style.background_gradient(cmap='bwr')

